In [ ]:
import numpy as np
import chainer
from chainer import cuda
import chainer.functions as F
from chainer import optimizers
import time
import pylab
import matplotlib.pyplot as plt
import pickle
import csv

with open('/.pkl','rb') as f1:#訓練データ読み込み
    dataset1 = pickle.load(f1)
    
with open('/.pkl','rb') as f2:#テストデータ読み込み
    dataset2 = pickle.load(f2)
    
gpu_flag = 0
if gpu_flag >= 0:
    cuda.check_cuda_available()
xp = cuda.cupy if gpu_flag >= 0 else np


train_set_x, train_set_y = dataset1[0]
train_set_y = [i[0] for i in train_set_y]
test_set_x, test_set_y = dataset2[0]

train_set_x = np.array(train_set_x).astype(xp.float32)
train_set_y = np.array(train_set_y).astype(xp.int32)
test_set_x = np.array(test_set_x).astype(xp.float32)
test_set_y = np.array(test_set_y).astype(xp.int32)

x_train = train_set_x.reshape(train_set_x.shape[0],-1)
y_train = train_set_y

x_test = test_set_x.reshape(test_set_x.shape[0],-1)
y_test = test_set_y

N = x_train.shape[0]
N_test = y_test.size

x_train = x_train.reshape((len(x_train), 3, 29, 29))
x_test = x_test.reshape((len(x_test), 3, 29, 29))
x_train = (x_train*1.0)/255
x_test = (x_test*1.0)/255

class AlexNet(chainer.Chain):#structure
    def __init__(self):
        super(AlexNet,self).__init__(
        conv1=F.Convolution2D(3, 32, 3),   
        conv2=F.Convolution2D(32, 64, 3), 
        conv3=F.Convolution2D(64, 64, 2),
        conv4=F.Convolution2D(64, 128, 2),
        conv5=F.Convolution2D(128, 128, 2),
        conv6=F.Convolution2D(128, 128, 2),
        conv7=F.Convolution2D(128, 256, 2),
        conv8=F.Convolution2D(256, 256, 2),
        conv9=F.Convolution2D(256, 256, 2),
        conv10=F.Convolution2D(256, 512, 2),
        conv11=F.Convolution2D(512, 512, 2),
        conv12=F.Convolution2D(512, 512, 2),
            l1=F.Linear(2048, 256),            
            l2=F.Linear(256, 2)
        )
        
    def __call__(self,x_data,train = True):
        x = chainer.Variable(x_data.astype(np.float32))
        h = F.max_pooling_2d(F.relu(self.conv1(x)), 2)
        h = F.relu(self.conv2(h))
        h = F.relu(self.conv3(h))
        h = F.relu(self.conv4(h))
        h = F.relu(self.conv5(h))
        h = F.relu(self.conv6(h))
        h = F.relu(self.conv7(h))
        h = F.relu(self.conv8(h))
        h = F.relu(self.conv9(h))
        h = F.relu(self.conv10(h))
        h = F.relu(self.conv11(h))
        h = F.relu(self.conv12(h))
        h = F.dropout(F.relu(self.l1(h)), train= train)
        y = self.l2(h)
        y = F.relu(y)
        return y

    def forward_Train(self, x_data,y_data):
        y = self(x_data ,True)
        t = chainer.Variable(y_data)
        return F.softmax_cross_entropy(y, t), F.accuracy(y,t)
    
    def forward_Test(self, x_data, y_data):
        t = chainer.Variable(y_data)
        y = self(x_data)
        return F.accuracy(y, t)   
        
    def forward_label(self, x_data,y_data):
        label_list = []
        t = chainer.Variable(y_data)
        y = self(x_data)
        for i in range(0,y.data.shape[0]):
            if y.data[i][0] > y.data[i][1]:
                label = 0
            else:
                label = 1 
            label_list = np.append(label_list,label) 

In [ ]:
batchsize = 500
n_epoch = 10

model = AlexNet()
optimizer = optimizers.Adam()
optimizer.setup(model)

if gpu_flag >= 0:
    cuda.get_device(gpu_flag).use()
    model.to_gpu()
    
# ---Train---
for epoch in range(1, n_epoch + 1):
    print "epoch: %d" % epoch

    perm = np.random.permutation(N)
    sum_loss = 0
    train_accuracy = 0
    for i in range(0, N, batchsize):
        x_batch = xp.asarray(x_train[perm[i:i + batchsize]],dtype=np.float32)
        y_batch = xp.asarray(y_train[perm[i:i + batchsize]],dtype=np.int32)
        
        model.zerograds()
        loss,train_acc = model.forward_Train(x_batch, y_batch)
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(y_batch)
        train_accuracy += float(train_acc.data) * len(y_batch)
        
        train_result = train_accuracy / N
        result = sum_loss / N

    print "train mean loss: %f" %(result)
    print "train accuracy: %f" %(train_result)


In [ ]:
#---Test---

f = open('/.csv','w')#test結果の保存
Write = csv.writer(f, lineterminator='\n')

start_time = time.clock()
min_result = 1.000
acc_label2 = []
max_acc=0.0
sum_accuracy = 0

for i in range(0, N_test, batchsize):
    x_batch = xp.asarray(x_test[i:i + batchsize])
    y_batch = xp.asarray(y_test[i:i + batchsize])
    acc = model.forward_Test(x_batch, y_batch)
    sum_accuracy += float(acc.data) * len(y_batch)
test_result = sum_accuracy / N_test
print "test accuracy: %f" % (test_result)

acc_label = []
min_label = []
for i in range(0, N_test, batchsize):
    x_batch = xp.asarray(x_test[i:i + batchsize])
    y_batch = xp.asarray(y_test[i:i + batchsize])

    acc = model.forward_label(x_batch,y_batch)
    acc_label = np.append(acc_label,acc)

acc_label2.append(acc_label)
    
csv_label = []
csv_label = np.append(csv_label,acc_label2[0]).reshape(len(acc_label2[0]),1)

print("write start")
for i in range(len(csv_label)):
    Write.writerow([csv_label[i,0]])
f.close()
print("write finish")

end_time = time.clock()
print end_time - start_time
